Propaganda Detectors

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier,LogisticRegression
from sklearn import metrics
import nltk

from nltk.corpus import stopwords

from nltk.stem import PorterStemmer
import tensorflow as tf

from tensorflow.keras import layers
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [54]:
dataframe = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DM project/task1.train.txt', sep="\t", header=None, names=["text", "article_id", "label"])
dataframe.dropna()


,text,article_id,label
0,"Et tu, Rhody? A recent editorial in the Provi...",727600136,non-propaganda
1,A recent post in The Farmington Mirror — our t...,731714618,non-propaganda
2,"President Donald Trump, as he often does while...",731714635,non-propaganda
3,"February is Black History Month, and nothing l...",728627182,non-propaganda
4,"The snow was so heavy, whipped up by gusting w...",728627443,non-propaganda
...,...,...,...
35981,From The Telegraph: Towns in Brazil have beco...,785781276,propaganda
35982,The second episode of Consortium News on Flash...,785781322,propaganda
35983,"It is beginning. Actually, it’s been happenin...",785781455,propaganda
35984,Justin’s note: As regular Dispatch readers kno...,785781711,propaganda


In [0]:
y = dataframe['label']
X_train,X_test,y_train,y_test = train_test_split(dataframe['text'],y,test_size =0.1, stratify=y)

In [0]:
dataframe.groupby('label').count()

,text,article_id
label,,
non-propaganda,3205,3205
propaganda,394,394


In [0]:
ps = PorterStemmer()
nltk.download('stopwords')
def my_tokenizer(str_input):
  str_input = str_input.replace("!"," exclamationsign")
  str_input = str_input.replace("?"," questionmark")
  str_input = str_input.replace("'"," singlequote")
  tokens = str_input.split()
  words = [ps.stem(token) for token in tokens if token not in stopwords.words('english')]
  return words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english',tokenizer=my_tokenizer,lowercase=True,min_df = 3, max_df=0.8,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1)

In [50]:
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_train = tfidf_vectorizer.transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'thu', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [51]:
linear_clf = PassiveAggressiveClassifier(max_iter=200)
linear_clf.fit(tfidf_train, y_train)
pred = linear_clf.predict(tfidf_test)
score_pa_tfidf = metrics.accuracy_score(y_test, pred)
print("Accuracy:", score_pa_tfidf)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_pa = (2*P*R)/(P+R)
print("F1 score : ",F1_pa)

Accuracy: 0.9744373437065852
F1 score :  0.8763440860215053


In [0]:
mlp = MLPClassifier() 
mlp.fit(tfidf_train, y_train)
pred = mlp.predict(tfidf_test)
score = metrics.accuracy_score(y_test, pred)
score_mlp_tfidf1=round(score,3)
#print(score_knn_tfidf)
print("accuracy:   %0.3f" % score)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_mlp = (2*P*R)/(P+R)
print(F1_mlp)


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


accuracy:   0.967
0.8345120226308346


In [0]:
svc = LinearSVC(C=1.0) 
svc.fit(tfidf_train, y_train)
pred = svc.predict(tfidf_test)
score = metrics.accuracy_score(y_test, pred)
score_svc_tfidf1=round(score,3)
#print(score_knn_tfidf)
print("accuracy:   %0.3f" % score)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_svc = (2*P*R)/(P+R)
print(F1_svc)

accuracy:   0.970
0.8475524475524475


In [0]:
lr = LogisticRegression()
lr.fit(tfidf_train, y_train)
pred = lr.predict(tfidf_test)
score_lr_tfidf = metrics.accuracy_score(y_test, pred)
score_lr_tfidf = round(score_lr_tfidf,3)
print("accuracy:   %0.3f" % score_lr_tfidf)
# cm_lr_tfidf = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda'])
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_lr = (2*P*R)/(P+R)
print(F1_lr)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


accuracy:   0.943
0.6688206785137318


In [0]:
import pickle
pkl_filename = "pd_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(linear_clf, file)

In [0]:
pickle.dump(tfidf_vectorizer,open("vectorizer.pkl","wb"))

In [0]:
pickle.dump(tfidf_vectorizer.vocabulary_,open("feature.pkl","wb"))

In [0]:
corpus = open("untitled.txt")

In [0]:
linear_clf.predict(test)

array(['non-propaganda', 'non-propaganda', 'non-propaganda',
       'non-propaganda', 'non-propaganda', 'propaganda', 'non-propaganda',
       'non-propaganda', 'non-propaganda', 'non-propaganda',
       'non-propaganda', 'non-propaganda', 'non-propaganda',
       'non-propaganda', 'non-propaganda', 'non-propaganda',
       'non-propaganda', 'non-propaganda', 'non-propaganda',
       'non-propaganda', 'non-propaganda', 'non-propaganda',
       'non-propaganda', 'non-propaganda', 'non-propaganda',
       'non-propaganda', 'propaganda', 'non-propaganda', 'non-propaganda',
       'non-propaganda', 'non-propaganda', 'non-propaganda',
       'non-propaganda', 'non-propaganda', 'non-propaganda',
       'non-propaganda', 'non-propaganda', 'non-propaganda',
       'non-propaganda', 'non-propaganda', 'non-propaganda',
       'non-propaganda', 'non-propaganda', 'non-propaganda',
       'non-propaganda', 'non-propaganda', 'non-propaganda',
       'non-propaganda', 'non-propaganda', 'non-propagand

In [0]:
test = tfidf_vectorizer.transform(corpus)